In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# 创建示例数据
np.random.seed(42)
data = pd.DataFrame({
    "group": np.random.choice(["<11小时", ">11小时"], size=1000),
    "value": np.concatenate([
        np.random.normal(loc=10, scale=2, size=500),
        np.random.normal(loc=15, scale=3, size=500)
    ]),
    "category": np.random.choice(range(1, 20), size=1000)
})

plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']  # 使用中文字体，如 "SimHei" (黑体) 或 "Microsoft YaHei" (微软雅黑)
plt.rcParams['axes.unicode_minus'] = False 

# 绘制分组直方图
g = sns.FacetGrid(data, row="category", col="group", margin_titles=True, height=2, aspect=4)
g.map(plt.hist, "value", bins=20, color="skyblue", edgecolor="white")

# 调整布局
g.set_axis_labels("分布时长_小时", "count")
g.set_titles(row_template="{row_name}", col_template="{col_name}")
g.figure.suptitle("分组时长分布_按类别", y=1.02)
plt.show()  


In [ ]:
import pandas as pd
import plotly.graph_objects as go

def export_dataframe_to_image_v2(
    df, 
    output_path, 
    title="DataFrame Export", 
    image_size=(800, 560), 
    font_family="Arial",  # 确保兼容的字体
    font_size=18
):
    """
    将 pandas DataFrame 导出为图片，增强样式显示效果。

    参数：
    - df: pandas DataFrame，需要导出的数据
    - output_path: str，图片保存路径
    - title: str，图片标题
    - image_size: tuple，图片尺寸 (宽, 高)，默认 (800, 560)
    - font_family: str，字体样式，默认 "Arial"
    - font_size: int，字体大小，默认 18
    """
    # 配色方案
    header_color = ['#007BFF', "#007BFF", '#007BFF', '#007BFF', '#8081cf', '#8081cf', '#3cc08e', '#3cc08e']  # 表头背景色
    header_font_color = "white"  # 表头字体颜色
    cell_fill_colors = ["#F9F9F9", "#FFFFFF"]  # 单元格条纹背景
    cell_font_color = "#333333"  # 默认单元格字体颜色

    # 动态设置字体颜色：带 + 号的数字为红色
    font_colors = []
    for col in df.columns:
        col_colors = []
        for value in df[col]:
            if isinstance(value, str) and "+" in value:  # 判断是否包含 + 号
                col_colors.append("red")  # 红色
            else:
                col_colors.append(cell_font_color)  # 默认颜色
        font_colors.append(col_colors)
    
    # 创建表格
    table = go.Figure(data=[go.Table(
        columnwidth=[2, 1.5, 1, 1.2, 1, 1.2, 1, 1.2],  # 列宽设置
        header=dict(
            values=[f"<b>{col}</b>" for col in df.columns],
            fill_color=header_color,
            align="center",
            font=dict(family=font_family, size=font_size, color=header_font_color),
            line_color="white",  # 表头边框颜色
            height=40  # 增大表头高度
        ),
        cells=dict(
            values=[df[col] for col in df.columns],
            fill_color=[cell_fill_colors * (len(df) // 2 + 1)],
            align="center",
            font=dict(family=font_family, size=font_size - 2),
            line_color="#E5E5E5",  # 单元格边框颜色
            height=30,  # 增大单元格高度
            font_color=font_colors  # 动态设置字体颜色
        )
    )])
    print(f"字体颜色{font_colors}\n")
    print(f"cell颜色{[cell_fill_colors * (len(df) // 2 + 1)]}\n")
    print(f"表头颜色{header_color}\n")
    # 布局设置
    table.update_layout(
        title=dict(
            text=f"<b>{title}</b>",
            font=dict(family=font_family, size=font_size + 4, color="#007BFF"),
            x=0.5,
            xanchor="center"
        ),
        margin=dict(l=20, r=20, t=70, b=20),  # 边距优化
        paper_bgcolor="white",  # 背景色
        width=image_size[0], 
        height=image_size[1]
    )

    # 保存图片
    table.write_image(output_path, width=image_size[0], height=image_size[1], scale=3)
    print(f"图片已保存到: {output_path}")
    
# 示例
if __name__ == "__main__":
    data = {
        "Name": ["Alice", "Bob", "Charlie"],
        "Age": [25, 30, 35],
        "City": ["New York", "Los Angeles", "Chicago"],
        "Money": ["-95221", "+52000", "+561511"]  # 注意：这里需要是字符串形式
    }
    df = pd.DataFrame(data)

    export_dataframe_to_image_v2(
        data, 
        output_path="output_v2.png", 
        title="寄修数据实时汇报",
        image_size=(850, 460),
        font_family="Microsoft YaHei",  # 替换为兼容字体
        font_size=18
    )

In [2]:
from datetime import datetime, timezone

# 获取当前时间的 UTC 时间
now_utc = datetime.now(timezone.utc)
print(now_utc)
# 格式化为 ISO 8601 格式
iso_format = now_utc.isoformat(timespec='milliseconds').replace('+00:00', 'Z')

print("当前时间的 UTC 时间（ISO 格式）：", iso_format)


2025-01-09 12:43:22.651275+00:00
当前时间的 UTC 时间（ISO 格式）： 2025-01-09T12:43:22.651Z


In [14]:
import hashlib
import time
import uuid
from urllib.parse import quote
import requests
import pandas as pd

def generate_requrl(pageindex,conditions):

    """
    从 API 获取数据并转换为 DataFrame
    """
    # 基本参数
    tenant = "laifen"
    api_name = "api/vlist/ExecuteQuery"
    timestamp = str(int(time.time() * 1000))
    reqid = str(uuid.uuid1())
    appid = "AS_department"
    queryid = "38c53a54-813f-a0e0-0000-06f40ebdeca5"
    is_user_query = "true"
    is_preview = "false"
    pagesize = "5000"
    paging = "true"
    key = "u7BDpKHA6VSqTScpEqZ4cPKmYVbQTAxgTBL2Gtit"
    orderby = "createdon descending"
    additionalConditions = quote(conditions, safe='')

    args = [conditions, appid, orderby, pageindex, pagesize, paging, reqid, tenant, timestamp, is_preview, is_user_query, queryid, key]
    
    """
    生成签名
    """
    
    sign_str = "".join(args)
    sign = hashlib.sha256(sign_str.encode('utf-8')).hexdigest().upper()
    #构建 URL
    url = (
        f"https://ap6-openapi.fscloud.com.cn/t/{tenant}/open/{api_name}"
        f"?$tenant={tenant}&$timestamp={timestamp}&$reqid={reqid}&$appid={appid}"
        f"&queryid={queryid}&isUserQuery={is_user_query}&isPreview={is_preview}"
        f"&$pageindex={pageindex}&$pagesize={pagesize}&$paging={paging}"
        f"&$additionalConditions={additionalConditions}&$orderby={orderby}&$sign={sign}"
    )

    return url

def fetch_api_data(url):

    # 发送 GET 请求
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"API 请求失败，状态码: {response.status_code}")

    # 解析 JSON 数据
    data = response.json()
    entities = data["Data"]["Entities"]

    df = pd.DataFrame(entities)

    return df

def getneeddata(df):
    df = df.assign(
    产品类型=df["new_productmodel_id"].apply(lambda x: x.get("name", None)),
    产品名称=df["new_product_id"].apply(lambda x: x.get("name", None)),
    旧件签收时间=df["FormattedValues"].apply(lambda x: x.get("new_signedon", None)),
    检测时间=df["FormattedValues"].apply(lambda x: x.get("new_checkon", None)),
    申请类别=df["FormattedValues"].apply(lambda x: x.get("new_srv_rma_0.new_applytype", None)),
    一检时间=df["FormattedValues"].apply(lambda x: x.get("laifen_onechecktime", None)),
    维修完成时间=df["FormattedValues"].apply(lambda x: x.get("laifen_servicecompletetime", None)),
    质检完成时间=df["FormattedValues"].apply(lambda x: x.get("laifen_qualityrecordtime", None)),
    瑞云单号 = df['new_rma_id'].apply(lambda x: x.get('name', None)),
    分拣人员 = df['laifen_systemuser2_id'].apply(lambda x: x.get('name', None) if pd.notnull(x) else None),
    处理状态=df["FormattedValues"].apply(lambda x: x.get("new_srv_rma_0.new_status", None)), 
    旧件处理状态=df["FormattedValues"].apply(lambda x: x.get("new_returnstatus", None)), 
    检测结果=df["FormattedValues"].apply(lambda x: x.get("new_solution", None)),
    故障现象= df['new_error_id'].apply(lambda x: x.get('name', None) if pd.notnull(x) else None),
    发货时间 = df['new_deliveriedon'],
    一检人员 = df['laifen_systemuser_id'].apply(lambda x: x.get('name', None) if pd.notnull(x) else None),
    发货状态 = df['FormattedValues'].apply(lambda x: x.get('new_srv_rma_0.new_deliverstatus', None)),
    物流单号 = df['new_deliverylogisticsnumber'],
    产品序列号 = df['new_userprofilesn'],
    服务人员 = df['new_srv_workorder_1.new_srv_worker_id'].apply(lambda x: x.get('name', None) if pd.notnull(x) else None)

)
#    # 选择需要的列
    df = df[[ 
       '瑞云单号','产品类型', '产品名称', '处理状态', '旧件处理状态', '检测结果', '申请类别', '旧件签收时间',
       '检测时间', '一检时间', '维修完成时间', '质检完成时间', '故障现象','发货时间','发货状态',
       '一检人员','产品序列号','物流单号','分拣人员'
    ]]
    
    return df


pageindex = "1"
days = 15
conditions = f'{"new_signedon":"{days}"}'
url = generate_requrl(pageindex,conditions)
rs = requests.get(url)
count = rs.json()['Data']['TotalRecordCount']
print(f"最近{days}天签收业务量共{count}单,共{count//5000+2}页数据")
datas = []

for i in range(1, count//5000+2):
    url = generate_requrl(str(i),conditions)
    data = fetch_api_data(url)
    print(f"第{i}页数据已获取")
    datas.append(data)
    

df = pd.concat(datas, ignore_index=True)
df = getneeddata(df)
df.to_csv('df.csv',index=False)

最近五天签收业务量共38953单
第1页数据已获取
第2页数据已获取
第3页数据已获取
第4页数据已获取
第5页数据已获取
第6页数据已获取
第7页数据已获取
第8页数据已获取


In [3]:
import altair as alt
from vega_datasets import data

iris = data.iris()
iris.to_json(orient='records')

'[{"sepalLength":5.1,"sepalWidth":3.5,"petalLength":1.4,"petalWidth":0.2,"species":"setosa"},{"sepalLength":4.9,"sepalWidth":3.0,"petalLength":1.4,"petalWidth":0.2,"species":"setosa"},{"sepalLength":4.7,"sepalWidth":3.2,"petalLength":1.3,"petalWidth":0.2,"species":"setosa"},{"sepalLength":4.6,"sepalWidth":3.1,"petalLength":1.5,"petalWidth":0.2,"species":"setosa"},{"sepalLength":5.0,"sepalWidth":3.6,"petalLength":1.4,"petalWidth":0.2,"species":"setosa"},{"sepalLength":5.4,"sepalWidth":3.9,"petalLength":1.7,"petalWidth":0.4,"species":"setosa"},{"sepalLength":4.6,"sepalWidth":3.4,"petalLength":1.4,"petalWidth":0.3,"species":"setosa"},{"sepalLength":5.0,"sepalWidth":3.4,"petalLength":1.5,"petalWidth":0.2,"species":"setosa"},{"sepalLength":4.4,"sepalWidth":2.9,"petalLength":1.4,"petalWidth":0.2,"species":"setosa"},{"sepalLength":4.9,"sepalWidth":3.1,"petalLength":1.5,"petalWidth":0.1,"species":"setosa"},{"sepalLength":5.4,"sepalWidth":3.7,"petalLength":1.5,"petalWidth":0.2,"species":"setosa